# Problem 1

* Current Stock Price 151.03

* Strike Price 165

* Current Date 03/13/2022

* Options Expiration Date 04/15/2022

* Risk Free Rate of 4.25%

* Continuously Compounding Coupon of 0.53%

Implement the closed form Greeks for GBSM. Implement a finite difference derivative calculation.

Compare the values between the two methods for both a call and a put.

Implement the binomial tree valuation for American options with and without discrete dividends. Assume the stock above:

* Pays dividend on 4/11/2022 of $0.88

Calculate the value of the call and the put. Calculate the Greeks of each.

What is the sensitivity of the put and call to a change in the dividend amount?

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.stats import norm

In [2]:
S = 151.03
X = 165
r = 0.0425
coupon = 0.0053
b = r - coupon
sigma = 0.2
current_date = datetime(2022,3,13)
exp_date = datetime(2022,4,15)

In [3]:
def time_to_maturity(current_date, exp_date):
    ttm = (exp_date - current_date).days / 365
    return ttm

def cal_d1_d2(S, X, T, sigma, b):
    d1 = (np.log(S/X)+(b + (sigma**2)/2)*T)/(sigma* np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return d1, d2
T = time_to_maturity(current_date, exp_date)

In [4]:
def delta_gbsm(option_type, S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    if option_type == "Call":
        delta = np.exp((b-r)*T) * norm.cdf(d1)
    else:
        delta = np.exp((b-r)*T) * (norm.cdf(d1)-1)
    return delta

In [5]:
print('For the closed form greeks for GBSM:')
print("Call's Delta is ", delta_gbsm('Call', S, X, T, sigma, b, r))
print("Put's Delta is ", delta_gbsm('Put', S, X, T, sigma, b, r))

For the closed form greeks for GBSM:
Call's Delta is  0.08297130333914773
Put's Delta is  -0.9165496333661425


In [6]:
def gamma_gbsm(S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    gamma = (norm.pdf(d1)*np.exp((b-r)*T)) / (S * sigma *np.sqrt(T))
    return gamma

print("Call's Gamma is ", gamma_gbsm( S, X, T, sigma, b, r))
print("Put's Gamma is ", gamma_gbsm( S, X, T, sigma, b, r))

Call's Gamma is  0.016822916101852648
Put's Gamma is  0.016822916101852648


In [7]:
def vega_gbsm(S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    vega = S * np.exp((b-r)*T) * norm.pdf(d1) * np.sqrt(T)
    return vega
print("Call's Vega is ", vega_gbsm( S, X, T, sigma, b, r))
print("Put's Vega is ", vega_gbsm( S, X, T, sigma, b, r))

Call's Vega is  6.938710929513443
Put's Vega is  6.938710929513443


In [8]:
def theta_gbsm(option_type, S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    if option_type == "Call":
        theta = -(S*np.exp((b-r)*T)*norm.pdf(d1)*sigma)/(2*np.sqrt(T))\
                -(b-r)*S*np.exp((b-r)*T)*norm.cdf(d1)\
                -r*X*np.exp(-r*T)*norm.cdf(d2)
    else:
        theta = -(S*np.exp((b-r)*T)*norm.pdf(d1)*sigma)/(2*np.sqrt(T))\
                +(b-r)*S*np.exp((b-r)*T)*norm.cdf(-d1)\
                +r*X*np.exp(-r*T)*norm.cdf(-d2)
    return theta
print("Call's Theta is ", theta_gbsm( "Call", S, X, T, sigma, b, r))
print("Put's Theta is ", theta_gbsm( "Put", S, X, T, sigma, b, r))

Call's Theta is  -8.126522359668838
Put's Theta is  -1.9409914783019566


In [9]:
def rho_gbsm(option_type, S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    if option_type == "Call":
        rho = T*X*np.exp(-r*T)*norm.cdf(d2)
    else:
        rho = -T*X*np.exp(-r*T)*norm.cdf(-d2)
    return rho
print("Call's Rho is ", rho_gbsm( "Call", S, X, T, sigma, b, r))
print("Put's Rho is ", rho_gbsm( "Put", S, X, T, sigma, b, r))

Call's Rho is  1.1025939156368187
Put's Rho is  -13.758003122735788


In [10]:
def carry_rho_gbsm(option_type, S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    if option_type == "Call":
        carry_rho = T*S*np.exp((b-r)*T)*norm.cdf(d1)
    else:
        carry_rho = -T*S*np.exp((b-r)*T)*norm.cdf(-d1)
    return carry_rho
print("Call's Carry Rho is ", carry_rho_gbsm( "Call", S, X, T, sigma, b, r))
print("Put's Carry Rho is ", carry_rho_gbsm( "Put", S, X, T, sigma, b, r))

Call's Carry Rho is  1.132953825011723
Put's Carry Rho is  -12.515271800549371


In [11]:
import inspect

# Implement a finite diference derivative calculation
def first_order_derivative(function, x, delta):
    result = (function(x+delta) - function(x-delta)) / (2*delta)
    return result

def second_order_devirvative(function, x, delta):
    result = (function(x+delta) + function(x-delta) - 2*function(x)) / (delta**2)
    return result

In [12]:
def cal_derivative_wrt_one(function, order, object_arg, delta = 1e-3):
    all_args = list(inspect.signature(function).parameters.keys())
    orders_dic = {1:first_order_derivative, 2:second_order_devirvative}

    def cal_derivative(*args, **kwargs):
        args_dic = dict(list(zip(all_args, args)) + list(kwargs.items()))
        value_arg = args_dic.pop(object_arg)

        def trans_into_one_arg(x):
            all_args = {object_arg:x, **args_dic}
            return function(**all_args)
        return orders_dic[order](trans_into_one_arg, value_arg, delta)
    return cal_derivative

In [13]:
def gbsm(option_type, S, X, T, sigma, r, b):
    d1 = (np.log(S/X)+(b + (sigma**2)/2)*T)/(sigma* np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "Call":
        call_value = S * np.exp((b-r)*T)*norm.cdf(d1) - X * np.exp(-r*T)*norm.cdf(d2)
        return call_value
    else:
        put_value = X * np.exp(-r*T)*norm.cdf(-d2) - S * np.exp((b-r)*T)*norm.cdf(-d1)
        return put_value

In [15]:
print("For finite difference:")
gbsm_delta = cal_derivative_wrt_one(gbsm, 1, 'S')
print("Call's Delta is ", gbsm_delta( "Call", S, X, T, sigma, r, b))
print("Put's Delta is ", gbsm_delta( "Put", S, X, T, sigma, r, b))
gbsm_gamma = cal_derivative_wrt_one(gbsm, 2 ,'S')
print("Call's Gamma is ", gbsm_gamma( "Call", S, X, T, sigma, r, b))
print("Put's Gamma is ", gbsm_gamma( "Put", S, X, T, sigma, r, b))
gbsm_vega = cal_derivative_wrt_one(gbsm, 1 ,'sigma')
print("Call's Vega is ", gbsm_vega( "Call", S, X, T, sigma, r, b))
print("Put's Vega is ", gbsm_vega( "Put", S, X, T, sigma, r, b))
gbsm_theta = cal_derivative_wrt_one(gbsm, 1 ,'T')
print("Call's Theta is ", -gbsm_theta( "Call", S, X, T, sigma, r, b))
print("Put's Theta is ", -gbsm_theta( "Put", S, X, T, sigma, r, b))
gbsm_rho = cal_derivative_wrt_one(gbsm, 1 ,'r')
print("Call's Rho is ", gbsm_rho( "Call", S, X, T, sigma, r, b))
print("Put's Rho is ", gbsm_rho( "Put", S, X, T, sigma, r, b))
gbsm_carry_rho = cal_derivative_wrt_one(gbsm, 1 ,'b')
print("Call's Carry Rho is ", gbsm_carry_rho( "Call", S, X, T, sigma, r, b))
print("Put's Carry Rho is ", gbsm_carry_rho( "Put", S, X, T, sigma, r, b))

For finite difference:
Call's Delta is  0.08297130374668171
Put's Delta is  -0.9165496329472944
Call's Gamma is  0.016822911064195978
Put's Gamma is  0.016822951920403284
Call's Vega is  6.938653056250743
Put's Vega is  6.93865305626673
Call's Theta is  -8.126308803761084
Put's Theta is  -1.9407779203106656
Call's Rho is  -0.030359909416688424
Put's Rho is  -1.2427313238703164
Call's Carry Rho is  1.1329550097096686
Put's Carry Rho is  -12.515270634423814


In [16]:
# No dividend binomial tree
def bt_no_div(call, underlying, strike, ttm, rf, b, ivol, N):
    dt = ttm/N
    u = np.exp(ivol*np.sqrt(dt))
    d = 1/u
    pu = (np.exp(b*dt)-d)/(u-d)
    pd = 1.0-pu
    df = np.exp(-rf*dt)
    z = 1 if call else -1

    def nNodeFunc(n):
        return (n+1)*(n+2) // 2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
    nNodes = nNodeFunc(N)

    optionValues = [0.0] * nNodes

    for j in range(N,-1,-1):
        for i in range(j,-1,-1):
            idx = idxFunc(i,j)
            price = underlying*u**i*d**(j-i)
            optionValues[idx] = max(0,z*(price-strike))

            if j < N:
                optionValues[idx] = max(optionValues[idx], df*(pu*optionValues[idxFunc(i+1,j+1)] + pd*optionValues[idxFunc(i,j+1)]))

    return optionValues[0]

In [17]:
from typing import List

def bt_with_div(call: bool, underlying: float, strike: float, ttm: float, rf: float, b:float, divAmts: List[float], divTimes: List[int], ivol: float, N: int):
    # Actually b = rf in discrete dividend condition
    # if there are no dividends or the first dividend is outside out grid, return the standard bt_american value
    if not divAmts or not divTimes or divTimes[0] > N:
        return bt_no_div(call, underlying, strike, ttm, rf, b, ivol, N)

    dt = ttm / N
    u = np.exp(ivol * np.sqrt(dt))
    d = 1 / u
    pu = (np.exp(b * dt) - d) / (u - d)
    pd = 1 - pu
    df = np.exp(-rf * dt)
    z = 1 if call else -1

    def nNodeFunc(n: int) -> int:
        return int((n + 1) * (n + 2) / 2)

    def idxFunc(i: int, j: int) -> int:
        return nNodeFunc(j - 1) + i

    nDiv = len(divTimes)
    nNodes = nNodeFunc(divTimes[0])

    optionValues = [0] * nNodes

    for j in range(divTimes[0], -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i, j)
            price = underlying * (u ** i) * (d ** (j - i))

            if j < divTimes[0]:
                #times before the dividend working backward induction
                optionValues[idx] = max(0, z * (price - strike))
                optionValues[idx] = max(optionValues[idx], df * (pu * optionValues[idxFunc(i + 1, j + 1)] + pd * optionValues[idxFunc(i, j + 1)]))
            else:
                #time of the dividend
                valNoExercise = bt_with_div(call, price - divAmts[0], strike, ttm - divTimes[0] * dt, rf, b, divAmts[1:], [t - divTimes[0] for t in divTimes[1:]], ivol, N - divTimes[0])
                valExercise = max(0, z * (price - strike))
                optionValues[idx] = max(valNoExercise, valExercise)

    return optionValues[0]

In [18]:
N = 200
div = [0.88]
div_date = datetime(2022, 4, 11)
div_time = [round((div_date - current_date).days / (exp_date - current_date).days * N)]

In [19]:
print("Using binomial tree")
print("For the condition without dividend:")
print("The value of call option is ", bt_no_div(True, S, X, T, r, b, sigma, N))
print("The value of put option is ", bt_no_div(False, S, X, T, r, b, sigma, N))
b = 0.0425
print("For the condition with dividend:")
print("The value of call option is ", bt_with_div(True, S, X, T, r, b, div, div_time, sigma, N))
print("The value of put option is ", bt_with_div(False, S, X, T, r, b, div, div_time, sigma, N))

Using binomial tree
For the condition without dividend:
The value of call option is  0.33600393488438995
The value of put option is  14.036960189311927
For the condition with dividend:
The value of call option is  0.2986182507141458
The value of put option is  14.556578296240984


In [20]:
print("Using binomial tree, calculate the greeks:")
bt_delta = cal_derivative_wrt_one(bt_with_div, 1, 'underlying')
print("Call's Delta is ", bt_delta(True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Delta is ", bt_delta(False, S, X, T, r, b, div, div_time, sigma, N))

bt_gamma = cal_derivative_wrt_one(bt_with_div, 2 ,'underlying')
print("Call's Gamma is ", bt_gamma( True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Gamma is ", bt_gamma( False, S, X, T, r, b, div, div_time, sigma, N))

bt_vega = cal_derivative_wrt_one(bt_with_div, 1 ,'ivol')
print("Call's Vega is ", bt_vega( True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Vega is ", bt_vega( False, S, X, T, r, b, div, div_time, sigma, N))

bt_theta = cal_derivative_wrt_one(bt_with_div, 1 ,'ttm')
print("Call's Theta is ", -bt_theta( True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Theta is ", -bt_theta( False, S, X, T, r, b, div, div_time, sigma, N))

bt_rho = cal_derivative_wrt_one(bt_with_div, 1 ,'rf')
print("Call's Rho is ", bt_rho( True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Rho is ", bt_rho( False, S, X, T, r, b, div, div_time, sigma, N))

bt_carry_rho = cal_derivative_wrt_one(bt_with_div, 1 ,'b')
print("Call's Carry Rho is ", bt_carry_rho( True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Carry Rho is ", bt_carry_rho( False, S, X, T, r, b, div, div_time, sigma, N))

Using binomial tree, calculate the greeks:
Call's Delta is  0.07257286328979373
Put's Delta is  -0.9383141177661258
Call's Gamma is  -8.881784197001252e-10
Put's Gamma is  1.3287149158713873e-05
Call's Vega is  6.319443776511474
Put's Vega is  5.675482057655223
Call's Theta is  -7.467912305700292
Put's Theta is  -0.4489716873523619
Call's Rho is  -0.02437109966921258
Put's Rho is  -1.160866467771804
Call's Carry Rho is  0.9626647836451507
Put's Carry Rho is  -11.311095439340946


In [21]:
# Sensitivity of the put and call to a change in dividend amount
delta = 1e-3
div_up = [0.88 + delta]
div_down = [0.88 - delta]
call_up = bt_with_div(True, S, X, T, r, b, div_up, div_time, sigma, N)
call_down = bt_with_div(True, S, X, T, r, b, div_down, div_time, sigma, N)
call_sens_to_div_amount = (call_up - call_down) / (2*delta)

put_up = bt_with_div(False, S, X, T, r, b, div_up, div_time, sigma, N)
put_down = bt_with_div(False, S, X, T, r, b, div_down, div_time, sigma, N)
put_sens_to_div_amount = (put_up - put_down) / (2*delta)
print(f"Sensitivity to dividend amount: Call: {call_sens_to_div_amount:.3f}, Put: {put_sens_to_div_amount:.3f}")

Sensitivity to dividend amount: Call: -0.021, Put: 0.941


# Problem 2

Using the options portfolios from Problem3 last week (named problem2.csv in this week’s repo) and assuming :

* American Options

* Current Date 03/03/2023

* Current AAPL price is 165

* Risk Free Rate of 4.25%

* Dividend Payment of 1.00 on 3/15/2023

Using DailyPrices.csv. Fit a Normal distribution to AAPL returns – assume 0 mean return. Simulate AAPL returns 10 days ahead and apply those returns to the current AAPL price (above). Calculate Mean, VaR and ES.

Calculate VaR and ES using Delta-Normal.

Present all VaR and ES values a dollar loss, not percentages.

Compare these results to last week’s results.

# Problem 3

Use the Fama French 3 factor return time series (F-F_Research_Data_Factors_daily.CSV) as well as the Carhart Momentum time series (F-F_Momentum_Factor_daily.CSV) to fit a 4 factor model to the following stocks.

AAPL FB UNH MA
MSFT NVDA HD PFE
AMZN BRK-B PG XOM
TSLA JPM V DIS
GOOGL JNJ BAC CSCO

Fama stores values as percentages, you will need to divide by 100 (or multiply the stock returns by 100) to get like units.

Based on the past 10 years of factor returns, find the expected annual return of each stock.

Construct an annual covariance matrix for the 10 stocks.

Assume the risk free rate is 0.0425. Find the super efficient portfolio.
